In [1]:
import numpy as np
import matplotlib.pyplot as plt

import subprocess
import os
import json

In [2]:
e2e_folder = "../../../../test/e2e/"
output_folder = "./csv/"
img_folder = "./imgs/"

In [ ]:
test_states = 8
iterations = 2

timeout = 300

json_stats = dict()

for n_states in range(1, test_states + 1):
    print(f"---- Starting test of n_states {n_states} ----")

    json_stats[n_states] = []

    with open(f"{e2e_folder}networks/epoch.toml", "r") as src:
        with open(f"{e2e_folder}/networks/simple_modified.toml", "w") as dst:
            for l in src:
                if l.startswith("n_states"):
                    dst.write(f"n_states = {n_states}\n")
                else:
                    dst.write(l)
    
    with open(f"benchmark.txt", "w") as f:
        f.write("")
    
    success = 0
    for i in range(iterations):
        try:
            subprocess.run([f"{e2e_folder}/build/runner", "-f", f"{e2e_folder}/networks/simple_modified.toml", "benchmark"], timeout=timeout)
            success += 1
        except:
            print(f"Timing out after {timeout}s")
            if success == 0 and i == iterations - 1:
                i -= 1

    with open(f"benchmark.txt", "r") as f:
        for l in f:
            json_stats[n_states].append(l)

    print("Updated json_stats")
    print(json_stats)

with open(f"{output_folder}/benchmark_total.txt", "w") as f:
    f.write(json.dumps(json_stats))

---- Starting test of n_states 1 ----
I[2025-08-24|18:59:29.600] Removing Docker containers and networks      
I[2025-08-24|18:59:29.658] cleanup dir                                  msg="Removing testnet directory `../../../../test/e2e//networks/simple_modified`"
I[2025-08-24|18:59:33.528] setup                                        msg="Generating testnet files in `../../../../test/e2e//networks/simple_modified`"
I[2025-08-24|18:59:34.188] load                                         msg="Starting transaction load (16 workers)..."
I[2025-08-24|18:59:34.188] Starting initial network nodes...            
E[2025-08-24|18:59:34.188] failed to send transaction                   err="failed request: Post \"http://127.0.0.1:5707/v1\": dial tcp 127.0.0.1:5707: connect: connection refused"
E[2025-08-24|18:59:34.189] failed to send transaction                   err="failed request: Post \"http://127.0.0.1:5704/v1\": dial tcp 127.0.0.1:5704: connect: connection refused"
D[2025-08-24|18:59:34.1

In [ ]:
with open(f"{output_folder}/benchmark_total.txt", "r") as f:
    json_stats = json.load(f)
print("Last json_stats")
print(json_stats)

In [ ]:
mean_times = dict()
max_times = dict()
min_times = dict()

for n_states, stats in json_stats.items():
    mean_times[n_states] = 0
    max_times[n_states] = -1000
    min_times[n_states] = 1000

    for json_stat in stats:
        stat = json.loads(json_stat)
        mean_times[n_states] += stat["mean"]
        max_times[n_states] = max(max_times[n_states], stat["max"])
        min_times[n_states] = min(min_times[n_states], stat["min"])
    mean_times[n_states] /= len(stats)

print(f"Stats for all {len(mean_times)} tests")
for n_states in mean_times:
    print(f"Stats for {n_states}: Mean {mean_times[n_states]} max {max_times[n_states]} min {min_times[n_states]}")

In [ ]:
# plotting inv_throughput

plt.plot(mean_times.keys(), mean_times.values(), marker='o')
plt.plot(max_times.keys(), max_times.values(), marker='o', color='orange')
plt.plot(min_times.keys(), min_times.values(), marker='o', color='green')
plt.fill_between(mean_times.keys(), min_times.values(), max_times.values(), color='gray', alpha=0.3)
plt.legend(["Mean", "Max", "Min"])

plt.xlabel("Number of states")
plt.ylabel("Block commit rate (s)")
plt.title("Block commit rate vs Number of states")
plt.xticks(range(1, len(mean_times) + 1), range(1, len(mean_times) + 1))
plt.grid()
plt.savefig(f"{img_folder}/block_commit_rate.png")
plt.show()

In [ ]:
throughput = dict()
for n_states in mean_times:
    throughput[int(n_states)] = 1 / mean_times[n_states]
    
plt.plot(throughput.keys(), throughput.values(), marker='o')

plt.xlabel("Number of states")
plt.ylabel("Throughput (blocks/s)")
plt.title("Throughput vs Number of states")
plt.xticks(range(0, len(throughput) + 1), range(0, len(throughput) + 1))
plt.xlim(-0.5, len(throughput) + 1)
plt.ylim(0, max(throughput.values()) * 1.1)
plt.grid()
plt.savefig(f"{output_folder}/throughput_vs_nstates.png")
plt.show()